# Contents

- [Privileged administration](#privileged-administration)
  - [Limit the number of Global Administrators to less than 5 (*high*)](##limit-the-number-of-global-administrators-to-less-than-5)
  - [Synchronized accounts (*high*)](##synchronized-accounts)
  - [Use groups for Azure AD role assignments (*high*)](##use-groups-for-azure-ad-role-assignments)
  - [PIM Alerts (*high*)](##pim-alerts)
  - [Recurring access reviews (*high*)](##recurring-access-reviews)
  - [Access Reviews: Enabled for all groups (*high*)](##access-reviews:-enabled-for-all-groups)
  - [Apps and Owners Can Change All Group Membership](##apps-and-owners-can-change-all-group-membership)
- [External Identities](#external-identities)
  - [Guest Invite Settings (*high*)](##guest-invite-settings)
  - [Guest User Access Restrictions (*high*)](##guest-user-access-restrictions)
- [User Setting](#user-setting)
  - [User role permissions (Application registration) (*high*)](##user-role-permissions-application-registration)
- [Custom Domains](#custom-domains)
  - [Verified Domains (*high*)](##verified-domains)
- [Enterprise Applications](#enterprise-applications)
  - [User Consent for Apps (*medium*)](##user-consent-for-apps)
  - [Group Owner Consent (*medium*)](##group-owner-consent)
  - [Application Owners (*high*)](##application-owners)
- [Conditional Access Policies (*high*)](#conditional-access-policies)
  - [Block Legacy Protocols (*high*)](##block-legacy-protocols)
  - [Require MFA for Administrators (*high*)](##require-mfa-for-administrators)
  - [Require MFA for Azure Management (*high*)](##require-mfa-for-azure-management)
  - [Restricted Locations (*medium*)](##restricted-locations)
  - [Require devices to be marked as compliant (*high*)](##require-devices-to-be-marked-as-compliant)

# Initialization

In [ ]:
# 8ed476fd-17af-4e67-bfee-9ddc23573f9d is my demo tenant
$TenantId = '8ed476fd-17af-4e67-bfee-9ddc23573f9d' #"<TENANT_ID>"

# connect once for all necessary scopes for this notebook - these are delegated permissions so we cannot do something the authenticated user could not already do!
# Disconnect-Graph
# TODO: use Get-MgContext to check if we need to connect again
# NOTE: Never request a write scope!
$Scopes = "Directory.AccessAsUser.All", "Policy.Read.All", "RoleManagement.ReadWrite.Directory", "RoleManagementAlert.Read.Directory", "AccessReview.Read.All", "Application.Read.All", "Directory.Read.All"
$null = Connect-MgGraph -Scopes $Scopes -TenantId $TenantId -ErrorAction Stop

Write-Host "Connected to tenant '$TenantId' with the following scope: $Scopes"

# load functions
. "..\src\functions.ps1"

# Privileged administration

## Limit the number of Global Administrators to less than 5

*Severity*: High

*Guid*: 9e6efe9d-f28f-463b-9bff-b5080173e9fe

[Entra ID best practice](https://learn.microsoft.com/en-us/azure/active-directory/roles/best-practices#5-limit-the-number-of-global-administrators-to-less-than-5)

*As a best practice, Microsoft recommends that you assign the Global Administrator role to fewer than five people in your organization...*

In [ ]:
$Setting = Get-EntraIdRoleAssignment -RoleName "Global Administrator"
$Compliant = $Setting.Count -lt 5

if($Compliant)
{
    Write-Host "Compliant to control; there are $($Setting.Count) Global Administrators (Assigned and Eligeble)" -ForegroundColor Green
}
else {
    Write-Host "Not compliant to control; there are $($Setting.Count) Global Administrators (Assigned and Eligeble)" -ForegroundColor Red
}

Write-Host "`nGlobal Administrators:`n"
$Setting | Select-Object -Property Id, @{ Name = 'displayName'; Expression = { $_.AdditionalProperties.displayName } }, @{ Name = 'userPrincipalName'; Expression = { $_.AdditionalProperties.userPrincipalName } } | Format-Table -AutoSize


## Synchronized accounts

*Severity*: High

*Guid*: 87791be1-1eb0-48ed-8003-ad9bcf241b99

Do not synchronize accounts with the highest privilege access to on-premises resources as you synchronize your enterprise identity systems with cloud directories.

If below list any users then `onPremisesSyncEnabled` is true (and their account is enabled). Those should have the role removed, and a cloud-only user created as a replacement.

[Entra ID best practice](https://learn.microsoft.com/en-us/azure/security/fundamentals/identity-management-best-practices#centralize-identity-management)

*Don’t synchronize accounts to Azure AD that have high privileges in your existing Active Directory instance...*

In [ ]:
$PrivilegedRolesList = @('62e90394-69f5-4237-9190-012177145e10','194ae4cb-b126-40b2-bd5b-6091b380977d','f28a1f50-f6e7-4571-818b-6a12f2af6b6c','29232cdf-9323-42fd-ade2-1d097af3e4de','b1be1c3e-b65d-4f19-8427-f6fa0d97feb9','729827e3-9c14-49f7-bb1b-9608f156bbb8','b0f54661-2d74-4c50-afa3-1ec803f12efe','fe930be7-5e62-47db-91af-98c3a49a38b1','c4e39bd9-1100-46d3-8c65-fb160da0071f','9b895d92-2cd3-44c7-9d02-a6ac2d5ea5c3','158c047a-c907-4556-b7ef-446551a6b5f7','966707d0-3269-4727-9be2-8c3a10f19b9d','7be44c8a-adaf-4e2a-84d6-ab2649e08a13','e8611ab8-c189-46e8-94e1-60213ab1f814')

$i = 0
$UsersWithPrivilegedRoles = $PrivilegedRolesList | ForEach-Object {    
    $RoleName = Get-MgBetaRoleManagementDirectoryRoleDefinition -UnifiedRoleDefinitionId $_ -Property "DisplayName" | Select-Object -ExpandProperty DisplayName
    $i += 1
    [int]$p = 100 * [float]$i / [float]($PrivilegedRolesList.Count)
    Write-Progress -Activity "Getting users with the role '$RoleName'" -PercentComplete $p -Status "$p% Complete"
    Get-EntraIdRoleAssignment -RoleName $RoleName | ForEach-Object {$_ | Add-Member -MemberType NoteProperty -Name 'Role' -Value $RoleName -PassThru -Force}
}

$UsersWithPrivilegedRoles | ForEach-Object {
    $UserWithPrivilegedRole = $_
    Get-MgUser -UserId $UserWithPrivilegedRole.Id -Property onPremisesSyncEnabled, DisplayName, AccountEnabled -ErrorAction SilentlyContinue | Select-Object -Property DisplayName, AccountEnabled, onPremisesSyncEnabled, @{ Name = 'Role';  Expression = {$UserWithPrivilegedRole.Role}}
} | Where-Object {$_.OnPremisesSyncEnabled -and $_.AccountEnabled} | Select-Object -ExcludeProperty OnPremisesSyncEnabled, AccountEnabled

## Use groups for Azure AD role assignments

*(WiP)*

*Severity*: High

*Guid*: e0d968d3-87f6-41fb-a4f9-d852f1673f4c

[Best Practice: Use groups for Azure AD role assignments and delegate the role assignment](https://learn.microsoft.com/en-us/azure/active-directory/roles/best-practices#6-use-groups-for-azure-ad-role-assignments-and-delegate-the-role-assignment)


In [ ]:
$RoleName = "Global Administrator"

# Get the directory role id for $RoleName
$DirectoryRoleId = Get-MgDirectoryRole -Filter "DisplayName eq '$RoleName'" | Select-Object -ExpandProperty Id
# Get currently assigned
$Assigned = Get-MgDirectoryRoleMember -DirectoryRoleId $DirectoryRoleId | Select-Object -ExpandProperty Id

# TODO: $Assigned includes eligeble that have activated the role, but does not provide any details. we need to kow the 'state' and if it is activated we can disregard

# Get the role definition id for $RoleName
$DirectoryRoleDefinitionId = Get-MgBetaRoleManagementDirectoryRoleDefinition -Filter "DisplayName eq '$RoleName'" -Property "id" | Select-Object -ExpandProperty Id
# get principals that are eligble for GA
$EligeblePrincipals = Get-MgBetaRoleManagementDirectoryRoleEligibilityScheduleInstance -Filter "roleDefinitionId eq '$DirectoryRoleDefinitionId'" | Select-Object -ExpandProperty PrincipalId

$DirectoryObjectByIds = $EligeblePrincipals # + $Assigned

$params = @{
    ids   = $DirectoryObjectByIds
    types = @(
        "user"
        "group"
    )
}

if($params.ids.Count -gt 0)
{
    $DirectoryObject = Get-MgDirectoryObjectById -BodyParameter $params

    $DirectoryObject | Select-Object Id, @{ Name = 'displayName'; Expression = { $_.AdditionalProperties.displayName } }, @{ Name = 'type'; Expression = { $_.AdditionalProperties.'@odata.type'.split('.') | Select-Object -Last 1 } }
}

## PIM Alerts

*Severity*: High

*Guid*: N/A

There should be no active alerts in PIM. If below identifies any active alerts go to [PIM alerts](https://portal.azure.com/#view/Microsoft_Azure_PIMCommon/ResourceMenuBlade/~/Alerts/resourceId//resourceType/tenant/provider/aadroles) for further details.

In [ ]:
$GovernanceRoleManagementAlerts = Get-MgBetaIdentityGovernanceRoleManagementAlert -Filter "scopeId eq '/' and scopeType eq 'DirectoryRole' and isActive eq true" -ExpandProperty "alertDefinition,alertConfiguration,alertIncidents"

$GovernanceRoleManagementAlerts | Select-Object -Property @{ Name = 'Alert'; Expression = { $_.alertDefinition.displayName } }, IncidentCount

We can also list affected principals. Note that in some cases there is no direct principal, ex. for the alert `NoMfaOnRoleActivationAlert`

In [ ]:
$GovernanceRoleManagementAlerts = Get-MgBetaIdentityGovernanceRoleManagementAlert -Filter "scopeId eq '/' and scopeType eq 'DirectoryRole' and isActive eq true" -ExpandProperty "alertDefinition,alertConfiguration,alertIncidents"

$GovernanceRoleManagementAlerts.alertIncidents.AdditionalProperties | Where-Object { $_.assigneeUserPrincipalName } | ForEach-Object {
    $_ | Select-Object -Property @{ Name = 'Role'; Expression = { $_.roleDisplayName } }, @{ Name = 'User'; Expression = { "$($_.assigneeDisplayName) ($($_.assigneeUserPrincipalName))" } }
}

## Recurring access reviews

*Severity*: High

*Guid*: eae64d01-0d3a-4ae1-a89d-cc1c2ad3888f

Configure recurring access reviews to revoke unneeded permissions over time.

[Best Practice: Configure recurring access reviews to revoke unneeded permissions over time](https://learn.microsoft.com/en-us/azure/active-directory/roles/best-practices#4-configure-recurring-access-reviews-to-revoke-unneeded-permissions-over-time)

If there are no access review definitions then there are no recurring access reviews.

In [ ]:
$AccessReviewDefinitions = Get-MgBetaIdentityGovernanceAccessReviewDefinition

Write-Host "Access review definitions: $(($AccessReviewDefinitions | Measure-Object).Count)"

## Access Reviews: Enabled for all groups

*Severity*: Medium

*Guid*: e6b4bed3-d5f3-4547-a134-7dc56028a71f

[Plan a Microsoft Entra access reviews deployment](https://learn.microsoft.com/en-us/azure/active-directory/governance/deploy-access-reviews)

## Apps and Owners Can Change All Group Membership

Chad Cox: Group Membership changes to all groups, this script list every role and member (not pim eligible) with this capability , every application with the permission, and every owner of the application. Some of the permissions are to unified and some are to security. either way can you imagine if someone granted access to a group that gave them all kinds of access to teams sites or access to other cloud resources.

https://www.linkedin.com/posts/chad-cox-194bb560_entraid-aad-azuread-activity-7093368251329495040-Ff9T

https://github.com/chadmcox/Azure_Active_Directory/blob/master/Applications/get-AppsandOwnersCanChangeAllGroupMembership.ps1

In [ ]:
#get the graph id
$Graph = Get-MgBetaServicePrincipal -filter "appId eq '00000003-0000-0000-c000-000000000000'" -ErrorAction Stop
#get the permission IDs
$group_permissions = $Graph | select -ExpandProperty approles | select * | where {$_.value -in ("GroupMember.ReadWrite.All","Group.ReadWrite.All")}


Get-MgBetaServicePrincipalAppRoleAssignedTo -ServicePrincipalId $graph.id -All | `
    where {$_.AppRoleId -in ($group_permissions.id)} | select PrincipalDisplayName, PrincipalId -Unique | foreach{
        Get-MgBetaServicePrincipal -serviceprincipalid $_.PrincipalId -ExpandProperty owners | foreach{
            $app = $null;$app = $_
            $app | select appid, displayname,PublisherName,@{N="via";Expression={"AppRoleAssignment"}}
            ($app.owners).id | select @{N="appid";Expression={$app.appid}}, `
                @{N="displayname";Expression={(Get-MgBetaDirectoryObjectById -Ids $_ | select -ExpandProperty AdditionalProperties | convertto-json | convertfrom-json).displayname}}, `
                @{N="PublisherName";Expression={$app.PublisherName}},@{N="via";Expression={"Owner of $($app.displayname)"}}
            (Get-MgBetaApplication -filter "appId eq '$($_.appid)'" -ExpandProperty owners | `
                select -expandproperty owners).id | select @{N="appid";Expression={$app.appid}}, `
                @{N="displayname";Expression={(Get-MgBetaDirectoryObjectById -Ids $_ | select -ExpandProperty AdditionalProperties | convertto-json | convertfrom-json).displayname}}, `
                @{N="PublisherName";Expression={$app.PublisherName}},@{N="via";Expression={"Owner of $($app.displayname)"}}
        }
    } | select DisplayName,via -Unique

$roles = '810a2642-a034-447f-a5e8-41beaa378541',',11451d60-acb2-45eb-a7d6-43d0f0125c13','45d8d3c5-c802-45c6-b32a-1d70b5e1e86e', `
  '744ec460-397e-42ad-a462-8b3f9747a02c', 'b5a8dcf3-09d5-43a9-a639-8e29ef291470', 'fdd7a751-b60b-444a-984c-02652fe8fa1c', `
  '69091246-20e8-4a56-aa4d-066075b2a7a8', 'f28a1f50-f6e7-4571-818b-6a12f2af6b6c', '29232cdf-9323-42fd-ade2-1d097af3e4de', `
  '9360feb5-f418-4baa-8175-e2a00bac4301', 'fe930be7-5e62-47db-91af-98c3a49a38b1', '62e90394-69f5-4237-9190-012177145e10'

  Get-MgBetaDirectoryRole -all | where {$_.RoleTemplateId -in $roles} -pv role | foreach{
    Get-MgBetaDirectoryRoleMember -DirectoryRoleId $_.Id | foreach{$_ | select -expandproperty AdditionalProperties | `
        convertto-json -Depth 5| convertfrom-json}  | select displayName, @{N="via";Expression={"Role Member of $($role.displayname)"}}
  } | select DisplayName,via -Unique

# External Identities

## Guest invite settings

*Severity*: High

*Guid*: be64dd7d-f2e8-4bbb-a468-155abc9164e9

External Collaboration Settings: Guest invite settings set to `'Only users assigned to specific admin roles can invite guest users'` or `'No one in the organization can invite guest users including admins (most restrictive)'`

In [ ]:
$AuthorizationPolicy = Get-MgPolicyAuthorizationPolicy

$Setting = $AuthorizationPolicy.AllowInvitesFrom
$Compliant = $Setting -in 'adminsAndGuestInviters', 'none'

if($Compliant)
{
    Write-Host "Compliant to control, setting is $($Setting)" -ForegroundColor Green
}
else {
    Write-Host "Not compliant to control, setting is $($Setting)" -ForegroundColor Red
}

## Guest user access restrictions

*Severity*: High

*Guid*: 459c373e-7ed7-4162-9b37-5a917ecbe48f

External Collaboration Settings: Guest user access set to `'Guest user access is restricted to properties and memberships of their own directory objects (most restrictive)'`

In [ ]:
# TODO: does not say anything about guest user access....

$ExternalIdentityPolicy = Get-MgBetaPolicyExternalIdentityPolicy #-ExpandProperty "AdditionalProperties"

# $ExternalIdentityPolicy | fl *
# $ExternalIdentityPolicy.AdditionalProperties | fl *



# User Setting

## User role permissions (Application registration)

*Severity*: High

*Guid*: a2cf2149-d013-4a92-9ce5-74dccbd8ac2a

Users can register applications should be set to `No`.

Users should not be allowed to register applications. Use specific roles such as `Application Developer`.

In [ ]:
$AuthorizationPolicy = Get-MgPolicyAuthorizationPolicy -Property "DefaultUserRolePermissions"

$Setting = $AuthorizationPolicy.DefaultUserRolePermissions.AllowedToCreateApps
$Compliant = $Setting -eq $false

if($Compliant)
{
    Write-Host "Compliant to control; users are not allowed to create applications" -ForegroundColor Green
}
else {
    Write-Host "Not compliant to control; users are allowed to create applications" -ForegroundColor Red
}

# Custom Domains

## Verified Domains

*Severity*: High

*Guid*: bade4aad-1e8c-439e-a946-667313c00567

Only validated customer domains are registered

In [ ]:
$Domains = Get-MgBetaDomain

$UnverifiedDomains = $Domains | Where-Object {-not $_.IsVerified}

$Setting = $UnverifiedDomains
$Compliant = $Setting.Count -eq 0

if($Compliant)
{
    Write-Host "Compliant to control; All ($($Domains.Count)) domains are verified" -ForegroundColor Green
}
else {
    Write-Host "Not compliant to control; There are unverified domains registered: $($Setting | Select-Object -ExpandProperty Id)" -ForegroundColor Red
}

# Enterprise Applications

##  User consent for apps

*Severity*: Medium

*Guid*: 459c373e-7ed7-4162-9b37-5a917ecbe48f

Consent & Permissions: Allow user consent for apps from verified publishers

[Configure how users consent to applications](https://learn.microsoft.com/en-us/azure/active-directory/manage-apps/configure-user-consent?pivots=ms-graph)

In [ ]:
$PolicyAuthorization = Get-MgPolicyAuthorizationPolicy #-ExpandProperty defaultUserRolePermissions
$permissionGrantPoliciesAssigned = $PolicyAuthorization.DefaultUserRolePermissions.permissionGrantPoliciesAssigned

$Setting = $permissionGrantPoliciesAssigned
$Compliant = $Setting[0] -ne "ManagePermissionGrantsForSelf.microsoft-user-default-legacy"

if($Compliant)
{
    Write-Host "Compliant to control; users are only allowed to consent to apps from verified publishers or not consent at all." -ForegroundColor Green
}
else {
    Write-Host "Not compliant to control; users are allowed to consent to all applications." -ForegroundColor Red
}

##  Group Owner Consent

*Severity*: Medium

*Guid*: 909aed8c-44cf-43b2-a381-8bafa2cf2149

Consent & Permissions: Allow group owner consent for selected group owners 

[Configure group owner consent to applications](https://learn.microsoft.com/en-us/azure/active-directory/manage-apps/configure-user-consent-groups?tabs=azure-portal)

In [ ]:
# TODO - example is using AzureADPreview module and we would like to stick to MS Graph

##  Application Owners

*Severity*: High

*Guid*: N/A

MITRE ATT&CK tactics: [Persistence](https://attack.mitre.org/tactics/TA0003/), [Privilege Escalation](https://attack.mitre.org/tactics/TA0004/)

Credit: [Chad Cox](https://github.com/chadmcox) / [Applications/get-BuiltinAPPOwners.ps1](https://github.com/chadmcox/Azure_Active_Directory/blob/master/Applications/get-BuiltinAPPOwners.ps1)

Read here how these can be exploited: [Azure AD privilege escalation - Taking over default application permissions as Application Admin](https://dirkjanm.io/azure-ad-privilege-escalation-application-admin/) - Note that the Owner of the service principal can exploit this in the same way, hence why we look for owners.

Below code snippets look for various applications that are at an increased risk from having owners. 

In [ ]:
Get-MgBetaServicePrincipal -all -ExpandProperty owners | `
    # looking for first-party Microsoft applications with owners
    # TODO: convert to filter
    where {$_.PublisherName -like "*Microsoft*" -or !($_.PublisherName -eq "Microsoft Accounts") -and $_.AppOwnerOrganizationId -eq 'f8cdef31-a31e-4b4a-93e4-5f571e91255a'} | `
        where {$_.owners -like "*"} | select appid, displayname,PublisherName, owners # TODO: look up owner

# TODO: can we look for anyone who persisted access through one of these?

Look for applications with application permission in Microsoft Graph and 1 or more owners assigned. Application permissions are often medium-high risk permissions.

In [ ]:
$MgGraphPermissionIds = Find-MgGraphPermission -All -PermissionType Application | Select-Object -ExpandProperty Id
$Applications = Get-MgBetaApplication -All -ExpandProperty Owners | Where-Object {
    $RequiredResourceAccess = $_.RequiredResourceAccess
    $ResourceAppId = $RequiredResourceAccess.ResourceAppId
    $ResourceAccessIds = $RequiredResourceAccess.ResourceAccess.Id
    $ResourceAppId -eq '00000003-0000-0000-c000-000000000000' -and $_.Owners.Count -gt 0 -and  ($ResourceAccessIds | Where-Object {$_ -In $MgGraphPermissionIds}).Count -gt 0
}

$Applications | Select-Object -Property DisplayName, AppId, Owners # TODO: look up owner

Look for applications with owners and any resource access that we do not consider low-risk. The applications listed below is worth looking into.

In [ ]:
# filter out some of the classical delegated low-risk permissions

$LowPermissions = @('14dad69e-099b-42c9-810b-d002981feec1', 'e1fe6dd8-ba31-4d61-89e7-88639da4683d', '64a6cdd6-aab1-4aaf-94b8-3cc8405e90d0', '7427e0e9-2fba-42fe-b0c0-848c9e6a8182', '37f7f235-527c-4136-accd-4a02d197296e')
# Find-MgGraphPermission -All | Where-Object {$_.Id -in $LowPermissions} # uncomment to list low-risk permissions
($LowPermissions | ForEach-Object {Find-MgGraphPermission $_}).Name
$Applications = Get-MgBetaApplication -All -ExpandProperty Owners | Where-Object {
    $RequiredResourceAccess = $_.RequiredResourceAccess
    $ResourceAppId = $RequiredResourceAccess.ResourceAppId
    $ResourceAccessIds = $RequiredResourceAccess.ResourceAccess.Id
    $HasOwners = $_.Owners.Count -gt 0
    $OnlyLowPermissions = $ResourceAppId -eq '00000003-0000-0000-c000-000000000000' -and ($ResourceAccessIds | Where-Object {$_ -In $LowPermissions}).Count -eq $LowPermissions.Count
    # Application has owners, has API permissions and those permission are not only low-risk permissions
    $HasOwners -and $null -ne $_.RequiredResourceAccess -and -not $OnlyLowPermissions
}

$Applications | Select-Object -Property DisplayName, AppId, Owners # TODO: look up owner

# Conditional Access Policies

## Block Legacy Protocols

*Severity*: High

*Guid*: 9e6efe9d-f28f-463b-9bff-b5080173e9fe

[Common Conditional Access policy: Block legacy authentication](https://learn.microsoft.com/en-us/azure/active-directory/conditional-access/howto-conditional-access-policy-block-legacy)

Below looks for a conditional access policy that blocks legacy protocols and also outputs users excluded.

In [ ]:
# we are looking for a policy that is enabled, the control is block, includes all users, and condition is legacy clients
$Filter = "state eq 'enabled' and grantControls/builtInControls/all(i:i eq 'block') and conditions/users/includeUsers/all(i:i eq 'All') and conditions/clientAppTypes/all(i:i eq 'exchangeActiveSync' or i eq 'other')"
# need to use the beta API as v1.0 does not include policies made from templates
$BlockLegacyProtocolPolicy = Get-MgBetaIdentityConditionalAccessPolicy -Filter $Filter

# $BlockLegacyProtocolPolicy | Select-Object -Property DisplayName, Id

$ExcludeUsers = $BlockLegacyProtocolPolicy.Conditions.Users.ExcludeUsers
$ExcludeGroups = $BlockLegacyProtocolPolicy.Conditions.Users.ExcludeGroups
$ExcludeGuestsOrExternalUsers = $BlockLegacyProtocolPolicy.Conditions.Users.ExcludeGuestsOrExternalUsers

# TODO:
# $ExcludeGroups
# $ExcludeGuestsOrExternalUsers

$Compliant = $null -ne $BlockLegacyProtocolPolicy -and $BlockLegacyProtocolPolicy.Count -gt 0

if($Compliant)
{
    Write-Host "Compliant to control; CA Policy found blocking legacy protocols" -ForegroundColor Green
}
else {
    Write-Host "Not compliant to control; No valid CA Policy found blocking legacy protocols" -ForegroundColor Red
}

$ExcludeUsers | Where-Object { $null -ne $_ } | ForEach-Object {
    $ExcludedUser = Get-MgUser -Filter "id eq '$_'"
    Write-Host "Excluded user: $($ExcludedUser.DisplayName) ($($ExcludedUser.UserPrincipalName))"
}

## Require MFA for Administrators

*Severity*: High

*Guid*: fe1bd15d-d2f0-4d5e-972d-41e3611cc57b

[Common Conditional Access policy: Require MFA for administrators](https://learn.microsoft.com/en-us/azure/active-directory/conditional-access/howto-conditional-access-policy-admin-mfa)

Below looks for a conditional access policy that matches the policy template `"Require multifactor authentication for admins"`

In [ ]:
# we are looking for a policy that is enabled, the control is MFA or authentication strenght, includes specific roles, and includes all applications
$PrivilegedRolesList = "('62e90394-69f5-4237-9190-012177145e10','194ae4cb-b126-40b2-bd5b-6091b380977d','f28a1f50-f6e7-4571-818b-6a12f2af6b6c','29232cdf-9323-42fd-ade2-1d097af3e4de','b1be1c3e-b65d-4f19-8427-f6fa0d97feb9','729827e3-9c14-49f7-bb1b-9608f156bbb8','b0f54661-2d74-4c50-afa3-1ec803f12efe','fe930be7-5e62-47db-91af-98c3a49a38b1','c4e39bd9-1100-46d3-8c65-fb160da0071f','9b895d92-2cd3-44c7-9d02-a6ac2d5ea5c3','158c047a-c907-4556-b7ef-446551a6b5f7','966707d0-3269-4727-9be2-8c3a10f19b9d','7be44c8a-adaf-4e2a-84d6-ab2649e08a13','e8611ab8-c189-46e8-94e1-60213ab1f814')"
# cannot filter on authenticationStrength: Invalid $filter: navigation property 'authenticationStrength' not found on type 'microsoft.graph.conditionalAccessPolicy'.
# we will do this when checking if the policy is compliant
$Filter = "state eq 'enabled' and conditions/applications/includeApplications/all(i:i eq 'All') and conditions/users/includeRoles/`$count gt 0 and conditions/users/includeRoles/all(i:i in $PrivilegedRolesList)" # and (grantControls/builtInControls/all(i:i eq 'mfa') or grantControls/authenticationStrength ne null)
# need to use the beta API as v1.0 does not include policies made from templates
$RequireMfaAdminsPolicy = Get-MgBetaIdentityConditionalAccessPolicy -Filter $Filter

# $RequireMfaAdminsPolicy | Select-Object -Property DisplayName, Id

$ExcludeUsers = $RequireMfaAdminsPolicy.Conditions.Users.ExcludeUsers
$ExcludeGroups = $RequireMfaAdminsPolicy.Conditions.Users.ExcludeGroups
$ExcludeGuestsOrExternalUsers = $RequireMfaAdminsPolicy.Conditions.Users.ExcludeGuestsOrExternalUsers

# TODO:
# $ExcludeGroups
# $ExcludeGuestsOrExternalUsers

if($RequireMfaAdminsPolicy.Count -gt 1)
{
    # $Compliant will become $false as we expect a single policy
    Write-Warning "Found multiple matching CA policies:`n$($RequireMfaAdminsPolicy.DisplayName -join ',')"
}
$Compliant = $null -ne $RequireMfaAdminsPolicy -and $RequireMfaAdminsPolicy.Count -eq 1 -and ('mfa' -in $RequireMfaAdminsPolicy.GrantControls.builtInControls -or $RequireMfaAdminsPolicy.GrantControls.authenticationStrength.requirementsSatisfied -eq 'mfa')

if($Compliant)
{
    Write-Host "Compliant to control; CA Policy found that requires administrators to use MFA or better" -ForegroundColor Green
}
else {
    Write-Host "Not compliant to control; No valid CA Policy found targeting Azure administrators" -ForegroundColor Red
}

if($Compliant)
{
    # only makes sense to show if the policy is compliant
    $ExcludeUsers | Where-Object { $null -ne $_ } | ForEach-Object {
        $ExcludedUser = Get-MgUser -Filter "id eq '$_'"
        Write-Host "Excluded user: $($ExcludedUser.DisplayName) ($($ExcludedUser.UserPrincipalName))"
    }
}

## Require MFA for Azure Management

*Severity*: High

*Guid*: 4a4b1410-d439-4589-ac22-89b3d6b57cfc

[Common Conditional Access policy: Require MFA for Azure management](https://learn.microsoft.com/en-us/azure/active-directory/conditional-access/howto-conditional-access-policy-azure-management)

Below looks for a conditional access policy that matches the policy template `"Require multifactor authentication for Azure management"`

In [ ]:
# we are looking for a policy that is enabled, the control is MFA or authentication strenght, targeting application "Microsoft Azure Management"
# cannot filter on authenticationStrength: Invalid $filter: navigation property 'authenticationStrength' not found on type 'microsoft.graph.conditionalAccessPolicy'.
# we will do this when checking if the policy is compliant
$Filter = "state eq 'enabled' and conditions/applications/includeApplications/all(i:i eq '797f4846-ba00-4fd7-ba43-dac1f8f63013') and conditions/users/includeRoles/`$count gt 0 and conditions/users/includeUsers/all(i:i eq 'All')" # and (grantControls/builtInControls/all(i:i eq 'mfa') or grantControls/authenticationStrength ne null)
# need to use the beta API as v1.0 does not include policies made from templates
$RequireMfaAdminsPolicy = Get-MgBetaIdentityConditionalAccessPolicy -Filter $Filter

# $RequireMfaAdminsPolicy | Select-Object -Property DisplayName, Id

$ExcludeUsers = $RequireMfaAdminsPolicy.Conditions.Users.ExcludeUsers
$ExcludeGroups = $RequireMfaAdminsPolicy.Conditions.Users.ExcludeGroups
$ExcludeGuestsOrExternalUsers = $RequireMfaAdminsPolicy.Conditions.Users.ExcludeGuestsOrExternalUsers

# TODO:
# $ExcludeGroups
# $ExcludeGuestsOrExternalUsers

if($RequireMfaAdminsPolicy.Count -gt 1)
{
    # $Compliant will become $false as we expect a single policy
    Write-Warning "Found multiple matching CA policies"
}
$Compliant = $null -ne $RequireMfaAdminsPolicy -and $RequireMfaAdminsPolicy.Count -eq 1 -and ('mfa' -in $RequireMfaAdminsPolicy.GrantControls.builtInControls -or $RequireMfaAdminsPolicy.GrantControls.authenticationStrength.requirementsSatisfied -eq 'mfa')

if($Compliant)
{
    Write-Host "Compliant to control; CA Policy found that requires MFA or better to use 'Microsoft Azure Management'" -ForegroundColor Green
}
else {
    Write-Host "Not compliant to control; No valid CA Policy found" -ForegroundColor Red
}

$ExcludeUsers | Where-Object { $null -ne $_ } | ForEach-Object {
    $ExcludedUser = Get-MgUser -Filter "id eq '$_'"
    Write-Host "Excluded user: $($ExcludedUser.DisplayName) ($($ExcludedUser.UserPrincipalName))"
}

## Restricted Locations

*Severity*: Medium

*Guid*: 079b588d-efc4-4972-ac3c-d21bf77036e5

[Using the location condition in a Conditional Access policy](https://learn.microsoft.com/en-us/azure/active-directory/conditional-access/location-condition)

Named locations can be used in numerous different way. A bad way to use them is to exclude from ex. enforcing MFA when coming from a `"trusted location"`. This does not conform to a `zero trust strategy`.

In [ ]:
# check if any named locations exist
$NamedLocation = Get-MgBetaIdentityConditionalAccessNamedLocation

$Compliant = $null -ne $NamedLocation -and $NamedLocation.Count -gt 0

if($Compliant)
{
    Write-Host "Compliant to control; $($NamedLocation.Count) Named Locations are defined" -ForegroundColor Green
}
else {
    Write-Host "Not compliant to control; No Named Locations are defined" -ForegroundColor Red
}

# we can look for policies that excludes locations under conditions but exclude those that block access
$Filter = "state eq 'enabled' and conditions/locations/excludeLocations/all(i:i ne null) and grantControls/builtInControls/all(i:i ne 'block')"
# need to use the beta API as v1.0 does not include policies made from templates
$PoliciesLocationExclusion = Get-MgBetaIdentityConditionalAccessPolicy -Filter $Filter

if($PoliciesLocationExclusion.Count -gt 0)
{
    Write-Warning "$($PoliciesLocationExclusion.Count) policies has location exclusions:"
    $PoliciesLocationExclusion | Select-Object -Property DisplayName, Id
}

## Require devices to be marked as compliant

*Severity*: High

*Guid*: 7ae9eab4-0fd3-4290-998b-c178bdc5a06c

[Require device to be marked as compliant](https://learn.microsoft.com/en-us/azure/active-directory/conditional-access/concept-conditional-access-grant#require-device-to-be-marked-as-compliant)

Requiring devices to be marked as compliant in CA policy grants can be a powerful way of ensuring that connections are made from devices that are managed by the organization. With sufficiently strict device configurations enforced, this can be combined with MFA, or just a standalone grant.

In [ ]:
# we are looking for a policy that is enabled, the control is require device to be marked as compliant
$Filter = "state eq 'enabled' and grantControls/builtInControls/`$count gt 0 and grantControls/builtInControls/all(i:i eq 'compliantDevice')"
# need to use the beta API as v1.0 does not include policies made from templates
$CompliantDevicePolicy = Get-MgBetaIdentityConditionalAccessPolicy -Filter $Filter

# $CompliantDevicePolicy | Select-Object -Property DisplayName, Id

$ExcludeUsers = $CompliantDevicePolicy.Conditions.Users.ExcludeUsers
$ExcludeGroups = $CompliantDevicePolicy.Conditions.Users.ExcludeGroups
$ExcludeGuestsOrExternalUsers = $CompliantDevicePolicy.Conditions.Users.ExcludeGuestsOrExternalUsers

# TODO:
# $ExcludeGroups
# $ExcludeGuestsOrExternalUsers

$Compliant = $null -ne $CompliantDevicePolicy -and $CompliantDevicePolicy.Count -gt 0

if($Compliant)
{
    Write-Host "Compliant to control; CA Policy found that requires devices to be marked as compliant:`n$($CompliantDevicePolicy.DisplayName -join ',')" -ForegroundColor Green
}
else {
    Write-Host "Not compliant to control; No valid CA Policy found" -ForegroundColor Red
}

$ExcludeUsers | Where-Object { $null -ne $_ } | ForEach-Object {
    $ExcludedUser = Get-MgUser -Filter "id eq '$_'"
    Write-Host "Excluded user: $($ExcludedUser.DisplayName) ($($ExcludedUser.UserPrincipalName))"
}